In [ ]:
require 'pco_api'

token = ENV['PCO_API_TOKEN']
secret = ENV['PCO_API_SECRET']

api = PCO::API.new(basic_auth_token: token, basic_auth_secret: secret)

require 'sqlite3'

# Open a database
db = SQLite3::Database.new "kids.db"

nil

## Load Service Types

In [ ]:
# db.execute <<-SQL
# drop table song_schedules;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table service_types (
    id varchar(20),
    name varchar(50)
  );
SQL

nil

In [ ]:
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |service_type|
      id = service_type['id']
      attrs = service_type['attributes']
      
      name = attrs['name']
      
      db.execute "insert into service_types values ( ?, ?)", [id, name]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end

In [ ]:
# Find a few rows
db.execute( "select * from service_types" ) do |row|
  p row
end

nil

## Load Plans

In [ ]:
# db.execute <<-SQL
# drop table song_schedules;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table plans (
    id varchar(20),
    service_type_id varchar(20),
    dates text
  );
SQL

nil

In [ ]:
require 'date'
db.execute( "delete from plans" )

# Load service types from PCO
db.execute( "select id from service_types" ) do |row|
  
  service_type_id = row[0]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types[service_type_id.to_i].plans.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |plan|
      id = plan['id']
      attrs = plan['attributes']
      rel = plan['relationships']
        
      dates = attrs['dates'] == 'No dates' ? nil : Date.strptime(attrs['dates'], '%B %d, %Y')
      short_dates = attrs['short_dates']
      
      db.execute "insert into plans values ( ?, ?, ?)", [id, service_type_id, dates.to_s]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

In [ ]:
# Find a few rows
db.execute( "select * from plans" ) do |row|
  p row
end

nil

## Load Scheduled People

In [ ]:
db.execute <<-SQL
drop table people;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table people (
    id varchar(20),
    first_name text,
    last_name text,
    status text,
    photo_url text,
    photo_thumbnail_url
  );
SQL

nil

In [ ]:

offset = 0
per_page = 100
fetch_more = true

while fetch_more

  response = api.services.v2.people.get(offset: offset, per_page: per_page)

  data = response["data"]

  data.each do |plan|
    id = plan['id']
    attrs = plan['attributes']
    rel = plan['relationships']
    
    first_name = attrs['first_name']
    last_name = attrs['last_name']
    status = attrs['status']
    photo_url = attrs['photo_url']
    photo_thumbnail_url = attrs['photo_thumbnail_url']
    
    db.execute "insert into people values ( ?, ?, ?, ?, ?, ?)",
      [id, first_name, last_name, status, photo_url, photo_thumbnail_url]
  end

  fetch_more = data.length == per_page
  offset = offset + per_page
end



nil

In [ ]:
# Find a few rows
db.execute( "select * from people" ) do |row|
  p row
end

nil

## Load People Emails

In [ ]:
# db.execute <<-SQL
# drop table person_emails;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table person_emails (
    id varchar(20),
    person_id varchar(20),
    address text,
    location text,
    "primary" integer
  );
SQL

nil

In [ ]:
#db.execute( "delete from person_emails" )

# Load service types from PCO
db.execute( "select id from people" ) do |row|
  
  person_id = row[0]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.people.v2.people[person_id.to_s].emails.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |plan|
      id = plan['id']
      attrs = plan['attributes']
      rel = plan['relationships']
      
      address = attrs['address']
      location = attrs['location']
      primary = attrs['primary']
      
      values = [id, person_id, address, location, primary ? 1 : 0]
        
      
      db.execute "insert into person_emails values ( ?, ?, ?, ?, ?)", values
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

In [ ]:
# Find a few rows
db.execute( "select * from person_emails" ) do |row|
  p row
end

nil

## Load Plan People

In [ ]:
# db.execute <<-SQL
# drop table plan_people;
# SQL

# Create table for arrangements
db.execute <<-SQL
  create table plan_people (
    id varchar(20),
    plan_id varchar(20),
    service_type_id varchar(20),
    person_id varchar(20),
    team_id varchar(20),
    team_position_name text,
    status char(1),
    prepare_notification int,
    notification_sent_at text
  );
SQL

nil

In [ ]:
#require 'date'
#db.execute( "delete from plan_people" )

# Load service types from PCO
db.execute( "select id, service_type_id from plans" ) do |row|
  
  plan_id = row[0]
  service_type_id = row[1]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types[service_type_id.to_i].plans[plan_id.to_i].team_members.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |plan_person|
      id = plan_person['id']
      attrs = plan_person['attributes']
      rel = plan_person['relationships']
    
      notification_sent_at = attrs['notification_sent_at']
      prepare_notification = attrs['prepare_notification'] ? 1 : 0
      status = attrs['status']
      team_position_name = attrs['team_position_name']
      
      person_id = rel.dig('person', 'data', 'id')
      team_id = rel.dig('team', 'data', 'id')
      
      values = [id, plan_id, service_type_id, person_id, team_id, team_position_name, status, prepare_notification, notification_sent_at]
      
      db.execute "insert into plan_people values ( ?, ?, ?, ?, ?, ?, ?, ?, ?)", values
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

In [ ]:
# Find a few rows
db.execute( "select * from plan_people" ) do |row|
  p row
end

nil

## Put it all together...

In [ ]:
query = <<-SQL
  SELECT DISTINCT
    people.first_name,
    people.last_name,
    pp.team_position_name,
    pp.status,
    pe.address
  FROM
    plan_people pp
    LEFT JOIN plans p on p.id = pp.plan_id
    LEFT JOIN people people on people.id = pp.person_id
    LEFT JOIN person_emails pe on pe.person_id = pp.person_id
  WHERE
    p.service_type_id = "645713"
    AND p.dates = "2018-09-02"
SQL

db.execute( query ) do |row|
  p row
end

nil